<a href="https://colab.research.google.com/github/ba88052/titanic-analysis/blob/main/Titanics_to_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##數據清理

In [107]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

url_train = "https://raw.githubusercontent.com/ba88052/titanic-analysis/main/Titanic_Data/train.csv"
url_test = "https://raw.githubusercontent.com/ba88052/titanic-analysis/main/Titanic_Data/test.csv"
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)
df_data = [df_train, df_test]
df_data = pd.concat(df_data)

In [108]:
#分出姓氏
def get_Surname(df):
    df['Surname'] = df.Name.str.split(', ', expand=True)[0]

##分出Title 並將Title分成 Officer Royalty Mr Mrs Miss Master

def get_titles(df):
    df['Title'] = df.Name.str.split(', ', expand=True)[1]
    df['Title'] = df.Title.str.split('.', expand=True)[0]

    Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Dr" : "Officer",
    "Rev": "Officer",
    "Jonkheer" : "Royalty",
    "Don" : "Royalty",
    "Sir" : "Royalty",
    "Lady" : "Royalty",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master"
}
    df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    df['Title'] = df.Title.map(Title_Dictionary)

#性別進行編碼
def get_sex(df):
    sex_encoding = {
        'male': 0,
        'female': 1
    }
    df['Sex'] = df['Sex'].map(lambda d: sex_encoding[d])

#加入Family_size欄位（為Parch + SibSp+1)，並區分家庭大小
def get_family_size(df):
    df["Family_size"] = df["Parch"] + df["SibSp"] +1
    df['Single'] = df['Family_size'].map(lambda s: 1 if s == 1 else 0)
    df['Small'] = df['Family_size'].map(lambda s: 1 if 2 <= s <= 4 else 0)
    df['Large'] = df['Family_size'].map(lambda s: 1 if 5 <= s else 0)

#cabin拉出是否有艙號
def get_cabin(df):
    df['Cabin'] = df['Cabin'].apply( lambda x: 1 if not pd.isnull(x) else 0 )
    df['Cabin'].unique()

#處理遺失值
def deal_num(df):
    df['Age'].fillna(df.groupby(['Sex', "Title"])["Age"].transform('median'),inplace=True)
    df["Embarked"].fillna("S", inplace=True)
    df["Fare"].fillna(df["Fare"].mean(), inplace = True)

def deal_with_data(df):
    get_titles(df)
    get_Surname(df)
    get_sex(df)
    get_cabin(df)
    get_family_size(df)
    deal_num(df)
    df = pd.get_dummies(df, columns = ["Embarked"])
    df = pd.get_dummies(df, columns = ["Title"])
    df = pd.get_dummies(df, columns = ["Surname"])
    df = df.drop(["Name", "Ticket", "Family_size","SibSp", "Parch"],axis=1)
    return df

In [109]:
df_data = deal_with_data(df_data)

In [110]:
#將train的data做切割，分成x和y
columns_X = set(df_data.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_data[0:891][columns_X]
train_y = df_data[0:891][columns_y]
test_X = df_data[891:][columns_X]

In [140]:
#因特徵量巨大，運用隨機森林，從模型中選擇較有用的特徵

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
clf = clf.fit(train_X, train_y)
features = pd.DataFrame()
features['feature'] = train_X.columns
features['importance'] = clf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)
model = SelectFromModel(clf, prefit=True, max_features=115)
train_reduced = model.transform(train_X)
test_reduced = model.transform(test_X)
train_reduced.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


(891, 115)

(891, 893)

In [ ]:
# find the best max_features
scores = {}
for i in range(115,350):
    model = SelectFromModel(clf, prefit=True, max_features=i)
    train_reduced = model.transform(train_X)
    score = compute_score(RFC, X=train_reduced, y=train_y, scoring='accuracy')
    scores[i] = score


In [139]:
scores1 = scores
max_key = max(scores1, key = scores1.get)
max_key,scores[max_key]

(115, 0.8316301550436258)

##使用的模型 RFC：

In [141]:
#使用的模型：
# 隨機森林參數：
# n_estimators: 樹的數量(default=10)。
# min_samples_leaf: 最終葉節點最少樣本數(default=1)；
#                   當樣本不大時，可不設定使用預設，若樣本數量非常大時，則推薦增加此參數值。
# min_samples_split:節點再劃分時所需的最小樣本數(default=2)；
#                   當樣本不大時，可不設定使用預設，若樣本數量非常大時，則推薦增加此參數值。
# oob_score: 是否採用袋外樣本(out-of-bag samples)來評估模型的準確度(default=False)。
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier( n_estimators = 460,
                              min_samples_split = 10,
                              min_samples_leaf = 1,
                              oob_score = True,
                              random_state = 1,)


In [142]:
# 內測分數
#將交叉驗證合併成函式reyurn mean()值
from sklearn.model_selection import cross_val_score

def compute_score(clf, X, y, scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = 5, scoring=scoring)
    return np.mean(xval)

score = compute_score(RFC, X=train_reduced, y=train_y, scoring='accuracy')
print ("CV score =" , score)
print ("****")

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

CV score = 0.8316301550436256
****


In [143]:
RFC.fit(train_reduced, train_y)
Ans = RFC.predict(test_reduced)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [144]:
Ansdf = pd.DataFrame()
Ansdf["PassengerId"] = test_X["PassengerId"]
Ansdf["Survived"] = Ans.astype(int)
Ansdf
Ansdf.to_csv(r'TitanicANS.csv',index=False)

#XGB

In [124]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()

In [128]:
xgbc.fit(train_X, train_y)
Ans = xgbc.predict(test_X)
Ansdf = pd.DataFrame()
Ansdf["PassengerId"] = test_X["PassengerId"]
Ansdf["Survived"] = Ans.astype(int)
Ansdf.to_csv(r'TitanicANS.csv',index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [129]:
xgbc.fit(train_reduced, train_y)
Ans1 = xgbc.predict(test_reduced)
Ansdf1 = pd.DataFrame()
Ansdf1["PassengerId"] = test_X["PassengerId"]
Ansdf1["Survived"] = Ans.astype(int)
Ansdf1.to_csv(r'TitanicANS1.csv',index=False)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
